In [2]:
using Luna

In [9]:
# MPC PARAMETERS
Nrt = 15                        # number of round trips
Npass = 2*Nrt                # number of passes

R = 200e-3                      # mirror radius of curvature [m]
k = Nrt-2                       # for k->Nrt MPC operation is closest to the stability edge and wm is maximized, reducing mirror damage 
L = R*(1-cos(pi*k/Nrt))         # MPC cell length [m] (see Anne-Lise Viotti, Multi-pass cells for post-compression of ultrashort laser pulses, 2022, https://doi.org/10.1364/OPTICA.449225)
C = L/R

mtype = :spherical              # mirror type, either :spherical or :parabolic
ϕm1 = [0,0,0,0]                 # mirror 1 dispersion expressed as a taylor expansion
ϕm2 = [0,0,0,0]                 # mirror 2 dispersion expressed as a taylor expansion

pres = 1.5 #1.5 #10e-10 #1.5    # gas pressure [bar]
gas = :Kr                       # gas type
ion = true                     # if true: enable ionisation response, if false: disable ionisation 
ion_model="PPT"                 # set to "ADK" or "PPT"; "ADK" is less accurate at low intensities but faster; "PPT" may crash at very high intensities

# PULSE PARAMETERS
λ0 = 1030e-9        # central wavelength [m]
τ = 150e-15         # pulse duration [s]; ignored if FROG spectrum is used
E_pulse = 250e-6    # pulse energy [J]
#w0 = 150e-6         # beam waist at focus [m]      # instead calculated later
M2 = 1.0           # beam quality                 # not yet implemented; how would you do that?
N0, n0, n2 = Tools.getN0n0n2(PhysData.wlfreq(λ0), gas; P=pres, T=PhysData.roomtemp)        # gas number density, linear and nonlinear refractive index [m^2/W]

(3.7181736576595142e25, 1.0005843204424483, 3.4199449874306914e-23)

In [5]:
println(L)
println(C)

0.38270909152852023
1.913545457642601


In [4]:
# # MPC PARAMETERS
# Nrt = 15                        # number of round trips
# Npass = 2 #2*Nrt                # number of passes

# R = 1                    # mirror radius of curvature [m]
# k = Nrt-2                       # for k->Nrt MPC operation is closest to the stability edge and wm is maximized, reducing mirror damage 
# L = 1.995 #R*(1-cos(pi*k/Nrt))         # MPC cell length [m] (see Anne-Lise Viotti, Multi-pass cells for post-compression of ultrashort laser pulses, 2022, https://doi.org/10.1364/OPTICA.449225)
# C = L/R

# mtype = :spherical              # mirror type, either :spherical or :parabolic
# ϕm1 = [0,0,0,0]                 # mirror 1 dispersion expressed as a taylor expansion
# ϕm2 = [0,0,0,0]                 # mirror 2 dispersion expressed as a taylor expansion

# pres = 1 #1.5 #10e-10 #1.5    # gas pressure [bar]
# gas = :Ar                       # gas type
# ion = false                     # if true: enable ionisation response, if false: disable ionisation 
# ion_model="PPT"                 # set to "ADK" or "PPT"; "ADK" is less accurate at low intensities but faster; "PPT" may crash at very high intensities

# # PULSE PARAMETERS
# λ0 = 1030e-9        # central wavelength [m]
# τ = 700e-15         # pulse duration [s]; ignored if FROG spectrum is used
# E_pulse = 7e-3    # pulse energy [J]
# #w0 = 150e-6         # beam waist at focus [m]      # instead calculated later
# M2 = 1.0           # beam quality                 # not yet implemented; how would you do that?
# N0, n0, n2 = Tools.getN0n0n2(PhysData.wlfreq(λ0), gas; P=pres, T=PhysData.roomtemp)        # gas number density, linear and nonlinear refractive index [m^2/W]

In [10]:
# n2 = 0.97e-23
P_peak = 0.94*E_pulse/τ             # peak power; 0.94 for gaussian pulses (see https://www.rp-photonics.com/peak_power.html)
P_crit = 3.77*λ0^2/(8*pi*n0*n2)     # critical power; above which beam collapses
σ = 1-P_peak/(sqrt(2)*P_crit)       # nonlinear correction factor that accounts for self focusing/Kerr lensing

0.761790828499522

In [15]:
b = 7*pi^2*n2*P_peak*k/(2*λ0^2)
println(b)

22.679444659135275


In [13]:
# Function to compute output wavelength bandwidth from input bandwidth and frequency-domain broadening factor
# Inputs:
#   b_omega: Broadening factor in frequency domain (Δω_out / Δω_in)
#   lambda0: Central wavelength [m]
#   delta_lambda_in: Input wavelength bandwidth [m]
#   accurate: If true, use exact calculation (better for large bandwidth)
# Output:
#   delta_lambda_out: Output wavelength bandwidth [m]

function compute_delta_lambda_out(b_omega, lambda0, delta_lambda_in; accurate=false)
    c = 299792458.0  # speed of light [m/s]

    if !accurate
        # Narrowband approximation
        return b_omega * delta_lambda_in
    else
        # Exact method assuming symmetric spectrum around lambda0
        lambda_min_in = lambda0 - delta_lambda_in / 2
        lambda_max_in = lambda0 + delta_lambda_in / 2

        delta_omega_in = 2 * pi * c * (1 / lambda_min_in - 1 / lambda_max_in)
        delta_omega_out = b_omega * delta_omega_in

        # Convert back to wavelength bandwidth assuming symmetric spectrum
        # Use derivative-based approximation for output bandwidth
        delta_lambda_out = (lambda0^2 / (2 * pi * c)) * delta_omega_out
        return delta_lambda_out
    end
end

# Example usage:
b_omega = b                      # Example broadening factor
lambda0 = 1030e-9                  # 1030 nm
delta_lambda_in = 17e-9            # 30 nm input bandwidth

println("Approximate Δλ_out: ", compute_delta_lambda_out(b_omega, lambda0, delta_lambda_in))
println("Accurate Δλ_out: ", compute_delta_lambda_out(b_omega, lambda0, delta_lambda_in; accurate=true))

Approximate Δλ_out: 3.855505592052997e-7
Accurate Δλ_out: 3.855768179715341e-7


In [19]:
w0_NL = sqrt(λ0*L/(2*pi)*sqrt(σ*(2*R/L-1)))   # beam waist in the focus in nonlinear regime
w0_L = sqrt(λ0*L/(2*pi) * sqrt(2*R/L-1))       # beam waist in the focus in linear regime  
wm = w0_L*sqrt(2*R/(2*R-L))                     # beam waist on the mirror in linear regime

println(w0_NL*1e6)
println(w0_L*1e6)
println(wm*1e6)

107.88455161259178
115.47828514537358
555.4198741358597


In [25]:
# Mode-matching for the beam
M_mirr = [1 0;-2*R 1];
M_prop2 = [1 -L/2;0 1];
M_prop1 = [1 1;0 1];
Mtot = M_prop2#*M_mirr*M_prop1;
A = Mtot[1,1];
B = Mtot[1,2];
C = Mtot[2,1];
D = Mtot[2,2];

q_0NL = 1/(0 + 1im*sqrt(σ)*λ0/(pi*w0_NL^2))
q_mNL = (A*q_0NL + B)/(C*q_0NL + D)
#q_mNL = (D*q_0NL - B)/(-C*q_0NL + A)

wm = sqrt(sqrt(σ)*λ0/pi * 1/imag(1/q_mNL))
Rm = n0/real(1/q_mNL)
println(wm*1e6)
println(Rm)

518.8960331584037
-0.2001168640884897


In [8]:
R

0.2

In [26]:
M_mirr = [1 0;-2*R 1];
M_prop2 = [1 -L/2;0 1];
M_prop1 = [1 1;0 1];
Mtot = M_prop1*M_mirr*M_prop2;
A = Mtot[1,1];
B = Mtot[1,2];
C = Mtot[2,1];
D = Mtot[2,2];

q_m = 1/(n0/-Rm + 1im*λ0/(pi*wm^2))

q_0L = (A*q_m + B)/(C*q_m + D)
w0_L = sqrt(λ0/(pi*imag(1/q_0L)))
R0 = n0/real(1/q_0NL)
println(w0_L*1e6)
println(R0)

2667.2374429529054
Inf


In [10]:
println(L)
println(R/2)
253/2

0.38270909152852023
0.1


126.5

### Calculate number of samples

In [11]:
λ0 = 1030e-9
τ = 150e-15
λlims = (600e-9, 1500e-9)
#trange = 15*τ
L = 300e-3

#grid = Grid.RealGrid(L, λ0, λlims, trange)

factors = [1, 2, 3, 5, 10, 15, 20, 30, 40, 50] #1:5:50

for factor in factors
    trange = factor .* τ
    grid = Grid.RealGrid(L, λ0, λlims, trange)
    println("factor = ", factor)
    println("Nt = ", length(grid.t))
    println("trange = ", (grid.t[end]-grid.t[1])*1e15, " fs")
    println("Nω = ", length(grid.ω))
    println("ωrange = ", (grid.ω[end]-grid.ω[1])*1e-15, " PHz")
    println("frange = ", (grid.ω[end]-grid.ω[1])*1e-15/2pi, " PHz")
    println("flims = ", PhysData.wlfreq(1500e-9)*1e-15./2pi , "-", PhysData.wlfreq(600e-9)*1e-15./2pi, " PHz")
    println("\n")
end

┌ Info: Freq limits 0.20 - 0.50 PHz
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:40
┌ Info: Samples needed: 449.69, samples: 512, δt = 333.56 as
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:44
┌ Info: Requested time window: 150.0 fs, actual time window: 170.8 fs
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:46


factor = 1
Nt = 256
trange = 170.11768855105754 fs
Nω = 129
ωrange = 4.709128918272134 PHz
frange = 0.7494811450000001 PHz
flims = 0.1998616386666667-0.4996540966666667 PHz


factor = 2
Nt = 512
trange = 340.9025052925114 fs
Nω = 257
ωrange = 4.709128918272134 PHz
frange = 0.7494811450000001 PHz
flims = 0.1998616386666667-0.4996540966666667 PHz


factor = 3
Nt = 1024
trange = 682.472138775419 fs
Nω = 513
ωrange = 4.709128918272134 PHz
frange = 0.7494811450000001 PHz
flims = 0.1998616386666667-0.4996540966666667 PHz


factor = 5
Nt = 2048
trange = 1365.6114057412344 fs
Nω = 1025
ωrange = 4.709128918272134 PHz
frange = 0.7494811450000001 PHz
flims = 0.1998616386666667-0.4996540966666667 PHz


factor = 10
Nt = 4096
trange = 2731.8899396728652 fs
Nω = 2049
ωrange = 4.709128918272134 PHz
frange = 0.7494811450000001 PHz
flims = 0.1998616386666667-0.4996540966666667 PHz


factor = 15
Nt = 4096
trange = 2731.8899396728652 fs
Nω = 2049
ωrange = 4.709128918272134 PHz
frange = 0.7494811450000001 

┌ Info: Grid: samples 256 / 512, ωmax 4.71e+15 / 9.42e+15
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:78
┌ Info: Freq limits 0.20 - 0.50 PHz
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:40
┌ Info: Samples needed: 899.38, samples: 1024, δt = 333.56 as
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:44
┌ Info: Requested time window: 300.0 fs, actual time window: 341.6 fs
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:46
┌ Info: Grid: samples 512 / 1024, ωmax 4.71e+15 / 9.42e+15
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:78
┌ Info: Freq limits 0.20 - 0.50 PHz
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:40
┌ Info: Samples needed: 1349.07, samples: 2048, δt = 333.56 as
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:44
┌ Info: Requested time window: 450.0 fs, actual time window: 683.1 fs
└ @ Luna.Grid C:\Users\mueld

In [12]:
λ0 = 1030e-9
pres = 1.5 #600e-3 #1.5 #10e-10 #1.5               # gas pressure [bar]
gas = :Kr #:Ar #:Kr           # gas type
N0, n0, n2 = Tools.getN0n0n2(PhysData.wlfreq(λ0), gas; P=pres, T=PhysData.roomtemp)        # gas number density, linear and nonlinear refractive index [m^2/W]
println("n2 = ", n2)

n2 = 3.4199449874306914e-23


### Calculate MPC parameters

In [13]:
# Creating an excel sheet for the L/R (also called C) values

# using XLSX

# # Create a dictionary to hold the calculated C values for each N
# C_values = Dict{Int, Vector{Float64}}()

# # Loop over each N from 1 to 30
# for N in 1:30
#     C_values[N] = [1 - cos(pi * k / N) for k in 1:(N-1)]
# end

# # Create an Excel workbook and add a worksheet
# XLSX.openxlsx("C_values.xlsx", mode = "w") do xf
#     sheet = xf[1]  # Access the first (and default) sheet
#     # Write headers for columns where k varies
#     for k in 1:29
#         sheet[1, k + 1] = "k=$k"
#     end

#     # Write the calculated values into the worksheet
#     row = 2
#     for N in 1:30
#         # Write the value of N in the first column
#         sheet[row, 1] = N
#         # Write the C values for this particular N starting from the second column
#         for (index, value) in enumerate(C_values[N])
#             sheet[row, index + 1] = value
#         end
#         row += 1
#     end
# end

# println("Excel sheet with C values has been created as 'C_values.xlsx'.")

In [26]:
λ0 = 1030e-9

Nrt = 15 #46/2 #5 #45/2 #15
k = Nrt-1
R = 200e-3 #1.5 #300e-3 #1.5 #200e-3 

println("R = ", R)

println("k\tL\tL/R")
for k in 1:Nrt-1
    L = R*(1-cos(pi*k/Nrt))
    C = L/R
    println("$k\t$L\t$C")
end

R = 0.2
k	L	L/R
1	0.004370479853238862	0.02185239926619431
2	0.017290908471479828	0.08645454235739913
3	0.03819660112501051	0.19098300562505255
4	0.06617387872822836	0.33086939364114176
5	0.09999999999999998	0.4999999999999999
6	0.1381966011250105	0.6909830056250524
7	0.17909430734646933	0.8954715367323466
8	0.22090569265353066	1.1045284632676533
9	0.2618033988749895	1.3090169943749475
10	0.3	1.4999999999999998
11	0.3338261212717716	1.6691306063588578
12	0.3618033988749895	1.8090169943749475
13	0.38270909152852023	1.913545457642601
14	0.39562952014676117	1.9781476007338057


In [32]:
C = 1.913545457642601
L = 0.38270909152852023	
w0 = sqrt(R*λ0/(2*pi) * sqrt(C*(2-C)))

0.00011547828514537356

In [33]:
zr = pi*w0^2/λ0

0.040673664307579994

In [36]:
Rz = L/2*(1+(zr/(L/2)^2))

0.40391110743428205

In [21]:
N0, n0, n2 = Tools.getN0n0n2(PhysData.wlfreq(1030e-9), :Kr; P=1.5, T=PhysData.roomtemp)

(3.7181736576595142e25, 1.0005843204424483, 3.4199449874306914e-23)

In [1]:
k=4
R = 300e-3
Nrt = 45/2
L = R*(1-cos(pi*k/Nrt))

0.04558557115307221

In [2]:
C = 551e-3/R

1.836666666666667

In [84]:
L/R

0.15195190384357404

In [58]:
Nrt = 15 #(45-1)/2
R = 200e-3 #1.5 #300e-3
k = Nrt-2
L = 379.9e-3 #R*(1-cos(pi*k/Nrt)) #1.99*R #551e-3 #1.99*R #L/R
C = L/R

1.8995

In [11]:
λ0 = 1030e-9
L = 1.99*1.5
R = 1.5
C = L/R

1.99

In [12]:
w0 = sqrt(R*λ0/(2*pi) * sqrt(C*(2-C)))
2*w0*1e3


0.3724925340125196

In [13]:
wm = sqrt(R*λ0/pi * sqrt(C/(2-C)))*1e6
2*wm*1e3


5.267839934832264e6

In [66]:
wm = sqrt(2*R*λ0*Nrt/pi^2)
wm*1e6

791.305828936984

In [166]:
5*wm

2.8715288728064796

In [3]:
λ0 = 1030e-9
gas = :Kr
pres = 1.5
N0, n0, n2 = Tools.getN0n0n2(PhysData.wlfreq(λ0), gas; P=pres, T=PhysData.roomtemp) 

(3.7181736576595142e25, 1.0005843204424483, 3.4199449874306914e-23)

In [4]:
1.984E-23*1.5

2.976e-23

### Calculate beam waists for linear mode matching

In [15]:
λ0 = 1030e-9
Nrt = 15
k = Nrt-1

R = 200e-3 #1.5
L = R*(1-cos(pi*k/Nrt))         # L = 290e-3 
C = L/R     #C = 1.9906859460363304 #1.986


w02 = R*λ0 * sqrt(C*(2-C)) / (2*pi)
w0 = sqrt(w02)
println("2*w0 = ", 2*w0.*1e6, " um")
println("w0 = ", w0.*1e6, " um")

wm2 = R*λ0 * sqrt(C/(2-C)) / pi
wm = sqrt(wm2)

# wm = w0*sqrt(2*R/(2*R-L))

println("2*wm = ", 2*wm.*1e6, " um")
println("wm = ", wm.*1e6, " um")

2*w0 = 165.12511261427778 um
w0 = 82.56255630713889 um
2*wm = 1579.714342412764 um
wm = 789.857171206382 um


### Calculate beam waists for nonlinear mode matching

In [3]:
# pulse parameters
λ0 = 1030e-9        # central wavelength [m]
τ = 150e-15         # pulse duration [s]; ignored if FROG spectrum is used
E_pulse = 250e-6    # pulse energy [J]
#w0 = 150e-6         # beam waist at focus [m]      # instead calculated later
M2 = 1.0           # beam quality                 # not yet implemented; how would you do that?

# MPC parameters
Nrt = 15           # number of round trips
Npass = 2*Nrt       # number of passes

R = 200e-3          # radius of curvature [m]
k = Nrt-2           # for k = Nrt-1 MPC operation is closest to the stability edge and wm is maximized reducing mirror damage 
#L = 380.1934e-3 #382.7090915285e-3 #395.6295201468e-3           # MPC cell length [m]
L = R * (1-cos(pi*k/Nrt))                                       # MPC cell length [m] calculated (see ref.[2])
C = L/R

propz = -L/2         # propagation distance from the waist [m] (distance between mirror position and focal position)
z_vals = L .* [0, 1/2, 1]     # points along the cell at which to investigate beam evolution [m]


pres = 1.5 #10e-10 #1.5               # gas pressure [bar]
gas = :Kr           # gas type
N0, n0, n2 = Tools.getN0n0n2(PhysData.wlfreq(λ0), gas; P=pres, T=PhysData.roomtemp)        # gas number density, linear and nonlinear refractive index [m^2/W]

P_peak = 0.94*E_pulse/τ             # peak power; 0.94 for gaussian pulses (see ref.[3])
P_crit = 3.77*λ0^2/(8*pi*n0*n2)     # critical power; above which beam collapses
σ = 1-P_peak/(sqrt(2)*P_crit)       # nonlinear correction factor that accounts for self focusing/Kerr lensing

0.761790828499522

In [4]:
println(P_peak)
println(P_crit)

1.5666666666666667e9
4.650537243721119e9


### Beam waists that will mode match the gas filled MPC due to nonlinear focusing:

In [17]:
w0_NL = sqrt(R*λ0 * sqrt(σ*C*(2-C)) /(2*pi))
println("w0_NL = ", w0_NL.*1e6, " um")
zr_NL = pi*w0_NL^2/λ0
propz = -L/2

wz_NL = w0_NL * sqrt(1+σ*(propz/zr_NL)^2)
println("wz_NL = ", wz_NL.*1e6, " um")

w0_NL = 107.88455161259176 um
wz_NL = 518.8960331584038 um


### Beam waist that has to be used as an input for the gaussgaussfield() function to end up with the nonlinear beam waist in the focus

In [18]:
propz = L/2

w0_L = sqrt(wz_NL^2 - sqrt(π^2 *wz_NL^4 - 4* λ0^2 *propz^2)/π)/sqrt(2)
# w0_L = sqrt(1+(propz/zr)^2) / wz
println("w0_L = ", w0_L.*1e6, " um")

w0_L = 124.54621284537734 um


# DAHER ET AL.


### Calculate beam waists for nonlinear mode matching

In [3]:
# pulse parameters
λ0 = 1030e-9        # central wavelength [m]
τ = 350e-15         # pulse duration [s]; ignored if FROG spectrum is used
E_pulse = 200e-6    # pulse energy [J]
#w0 = 150e-6         # beam waist at focus [m]      # instead calculated later
M2 = 1.0           # beam quality                 # not yet implemented; how would you do that?

# MPC parameters
Nrt = 5           # number of round trips
Npass = 2*Nrt       # number of passes

R = 300e-3          # radius of curvature [m]
k = Nrt-2           # for k = Nrt-1 MPC operation is closest to the stability edge and wm is maximized reducing mirror damage 
#L = 380.1934e-3 #382.7090915285e-3 #395.6295201468e-3           # MPC cell length [m]
L = 551e-3 #R * (1-cos(pi*k/Nrt))                                       # MPC cell length [m] calculated (see ref.[2])
C = L/R

propz = -L/2         # propagation distance from the waist [m] (distance between mirror position and focal position)
z_vals = L .* [0, 1/2, 1]     # points along the cell at which to investigate beam evolution [m]


pres = 1.0 #10e-10 #1.5               # gas pressure [bar]
gas = :Xe           # gas type
N0, n0, n2 = Tools.getN0n0n2(PhysData.wlfreq(λ0), gas; P=pres, T=PhysData.roomtemp)        # gas number density, linear and nonlinear refractive index [m^2/W]

P_peak = 0.94*E_pulse/τ             # peak power; 0.94 for gaussian pulses (see ref.[3])
P_crit = 3.77*λ0^2/(8*pi*n0*n2)     # critical power; above which beam collapses
σ = 1-P_peak/(sqrt(2)*P_crit)       # nonlinear correction factor that accounts for self focusing/Kerr lensing

0.8401948812938029

In [4]:
println(n2)
println(n0^2)

6.691396288277442e-23
1.0012675366791968


In [55]:
γ3 = PhysData.γ3_gas(gas)

9.544637297781407e-51

In [56]:
pres = 1.0
n2 = 5.2e-23 #6.691396288277442e-23
dens0 = PhysData.density(gas, pres)
χ3 = 4/3 * PhysData.ε_0*PhysData.c * n2 * n0^2
γ3 = χ3/dens0

7.436400870534807e-51

In [39]:
n2 = 5.2e-23 #6.691396288277442e-23
dens0 = PhysData.density(gas, pres)
Chi3 = 4/3 * PhysData.ε_0*PhysData.c * n2 # note: assuming n0 ≈ 1
γ3 = Chi3/dens0
println("γ3 = ",γ3)

γ3 = 7.407913225952237e-51


In [41]:
7.407913225952237e-51/2.4417194698078558e-51

3.033892024678487

In [10]:
w0 = sqrt(R*λ0/(2*pi) * sqrt(C*(2-C)) )

0.00016412154546772154

In [14]:
wm = sqrt(R*λ0/pi * sqrt(C/(2-C)) )

0.000574305774561296

In [ ]:
w0_NL = sqrt(λ0*L/(2*pi) * sqrt(σ*(2*R/L-1)))

0.00013908623153512486

In [12]:
zr_NL = pi*w0_NL^2/λ0
propz = -L/2

wz_NL = w0_NL * sqrt(1+σ*(propz/zr_NL)^2)
println("wz_NL = ", wz_NL.*1e6, " um")

wz_NL = 486.70042501093536 um


In [13]:
propz = L/2

w0_L = sqrt(wz_NL^2 - sqrt(π^2 *wz_NL^4 - 4* λ0^2 *propz^2)/π)/sqrt(2)
# w0_L = sqrt(1+(propz/zr)^2) / wz
println("w0_L = ", w0_L.*1e6, " um")

w0_L = 204.52086815397297 um
